# [E-04] AI_Lyricist
"Exploration Node 4. 멋진 작사가 만들기" / 2022. 01. 20 (Thu) 이형주

## Contents
---
- **데이터 준비 및 전처리**
- **NLP 모델 설계**
- **NLP 모델 평가**
- **프로젝트 회고**


## Rubric 평가기준
---

|  평가문항  |  상세기준  |
|:---------|:---------|
|1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?|텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
|2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?|특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
|3. 텍스트 생성모델이 안정적으로 학습되었는가?|텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

## 데이터 준비 및 전처리

**Step 1. 데이터를 불러와서** 전처리(Pre-Processing)를 준비합니다.
+ glob 모듈을 사용하여 파일을 불러오고, 문장 단위로 끊습니다.
+ 끊어진 문장들은 corpus(데이터의 한 뭉치)로 저장됩니다.

In [13]:
import os, re 
import glob
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

for txt_file in txt_list:
    # with구문으로 txt_list에 있는 txt_file 하나하나를 열어줍니다.
    with open(txt_file, "r") as f:
        # line을 기준으로 문장단위로 끊어서 raw변수에 넣어줍니다.
        raw = f.read().splitlines()
        # 리스트로 생성된 raw는 append가 아닌 extend로 raw_corpus에 더합니다.
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


**Step 2.** 데이터를 불러왔으니, **전처리를 시작합니다.**
+ 토큰의 갯수가 15개를 넘어가지 않도록 처리합니다.

In [2]:
raw_corpus[:15]

["Now I've heard there was a secret chord",
 'That David played, and it pleased the Lord',
 "But you don't really care for music, do you?",
 'It goes like this',
 'The fourth, the fifth',
 'The minor fall, the major lift',
 'The baffled king composing Hallelujah Hallelujah',
 'Hallelujah',
 'Hallelujah',
 'Hallelujah Your faith was strong but you needed proof',
 'You saw her bathing on the roof',
 'Her beauty and the moonlight overthrew her',
 'She tied you',
 'To a kitchen chair',
 'She broke your throne, and she cut your hair']

+ 불필요한 특수기호나, 공백을 문장에서 지웁니다. 
+ 하지만, 공백을 기준으로 단어를 토큰화 할 수 있도록 아래와 같이 처리합니다.

In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 문장 필터링 결과 예시
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


+ 문장 시작에는 'start', 문장이 끝날 때에는 'end'가 붙은 예시 10개가 아래와 같이 정상적으로 출력됩니다.

In [4]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜀
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 결과를 출력
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

+ 문장을 일정한 간격으로 쪼깨면, 여러개의 단어로 만들 수 있는데 **토큰화(Tokenize) 한다고 합니다.**
+ **이 토큰화 단어장의 크기를 12,000개까지 만들 것입니다.**
    - filters: 전처리 로직
    - oov -> out of vocabulary, 사전에 없는 단어는 unk로 처리
    - corpus -> Tokenizer가 단어 사전을 만들고, Tokenizer가 학습 모델에 입력할 데이터셋 Tensor로 변환
    - padding 메소드는 입력 데이터의 시퀀스 길이를 일정하게 맞춤

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences 사용

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줌
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,'\n', tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

len(tensor), len(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...    0    0    0]
 ...
 [   2  130    5 ...    0    0    0]
 [   2   23   89 ...    0    0    0]
 [   2    7   34 ...    0    0    0]] 


(175749, 175749)

+ 문장을 여러개의 단어로 쪼개고, **쪼갠 단어를 컴퓨터가 이해하도록 Tensor로 변환 되었습니다.**
+ '토근화'된 데이터를 보니, **모든 데이터가 2로 시작해서 3으로 끝나는 경향성**이 보입니다.
    - 정해진 입력 시퀀스 길이보다 짧은 경우 0으로 여백을 의미하는 패딩(padding)을 채워 넣었습니다.
    - 0은 패딩 문자 pad로도 불릴 수 있습니다.

In [6]:
print(tensor[:10, :12])

[[   2   50    5   91  297   65   57    9  969 6042    3    0]
 [   2   17 2639  873    4    8   11 6043    6  329    3    0]
 [   2   36    7   37   15  164  282   28  299    4   47    7]
 [   2   11  354   25   42    3    0    0    0    0    0    0]
 [   2    6 3604    4    6 2265    3    0    0    0    0    0]
 [   2    6 6044  292    4    6 1280  792    3    0    0    0]
 [   2    6 8541  523    1 1114 1114    3    0    0    0    0]
 [   2 1114    3    0    0    0    0    0    0    0    0    0]
 [   2 1114    3    0    0    0    0    0    0    0    0    0]
 [   2 1114   21  912   57  571   36    7  980 1845    3    0]]


+ 아래와 같이 확인해 보니, **2는 start이고 3은 end 입니다. 의도한 대로 데이터 가공(전처리)이 되었습니다.**

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: 
        break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


+ 데이터 가공은 되었으니, **학습할 데이터와 (처리 후)의 정답 데이터를 만듭니다.**
    - 학습할 데이터는 (x-data), 정답 데이터는 (y-data)가 됩니다.
    - DataSet 객체를 생성한 다음, 데이터 셋을 Train과 Test로 나눕니다.

In [8]:
# 소스 문장 : tensor에서 마지막 토큰(<end> or <pad>)을 제외.
src_input = tensor[:, :-1]  
# 타겟 문장 : tensor에서 <start>를 잘라내 생성.
tgt_input = tensor[:, 1:]   

print(src_input[0])
print(tgt_input[0])

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [28]:
## 데이터셋 객체 생성

BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

## tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
## enc_train + dec_train = dataset_train
## enc_val + dec_val = dataset_val
VOCAB_SIZE = tokenizer.num_words + 1   

dataset_train = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset_train = dataset_train.shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
dataset_train

dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)
dataset_val

<BatchDataset shapes: ((256, 346), (256, 346)), types: (tf.int32, tf.int32)>

## NLP 모델 설계

+ 10 Epoch(횟수) 내에 val_loss(오류) 값이 2.2 내외로 줄일 수 있도록 설계
+ 모델은 Embedding Layer, Lstm Layer, Dense Layer로 구성
    - Input을 모델에 투입시키면 Input Shape가 지정되고, model.build()가 호출

In [29]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 128
hidden_size = 512
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

+ 모델 테스트 적용 완료, Epoch 1회를 모델로 돌려봅니다.
+ matplotlib를 사용하여 Training Loss와 Validation Loss 결과를 시각화 합니다.

+ 메모리 부족으로 모델 가동에 실패했습니다.
    -  Batch Size 256, embedding_size, hidden_size가 각각 256, 1024로는 구동이 어렵게 되었습니다.
    - [Batch Size를 줄이면 대부분 해결된다](https://newindow.tistory.com/260)는 점을 알게 되었습니다.


+ Batch Size는 유지하고, 다른 사이즈들을 절반으로 줄여 1회로 Epoch 재시도 합니다.
    - batch size = 256 유지
    - embedding_size = 256에서
        + **embedding_size = 128으로**
    - hidden_size = 1024
        + **hidden_size = 512으로**

In [30]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset_train, epochs=1, validation_data=dataset_val)

549/549 [==============================] - 1035s 2s/step - loss: 0.2783 - val_loss: 0.1575


## NLP 모델 평가

+ 예시 문장 뒤에 나올 문장을 자동으로 생성하는 함수
    - Init_sentence -> Tokenizer -> Tensor 변환
    - While -> 단어 생성
    - end를 예측 하거나, max_len에 도달하면 종료. 예측한 단어는 입력한 문장 뒤에 추가됨

In [39]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=30):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:

        predict = model(test_tensor) 
        
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 

        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated  

In [40]:
generate_text(model, tokenizer, init_sentence="<start> i like", max_len=30)

'<start> i like t t the <end> '

## 프로젝트 회고

+ 문제로 제시한 Epoch, Hiperparameters 값들을 적용하였을 때 메모리 에러가 발생하였다.
    - 원인 해결에 많은 시간을 쏟았으나, 결국 수치를 낮춰 해결한 이유로 추후 고사양 데스크탑으로 로컬 환경에서 구현해보고 테스트를 진행해볼 예정이다.
+ 1회 가동만으로도 **Validation Loss가 이렇게 낮게 나오는 것은 Layer 적용에 문제가 있는 것이 아닐까** 생각된다.
    - 검색 결과 [bidirectional LSTM Layer를 쓰는 것이 더 효율적](https://hyen4110.tistory.com/29)일 것으로 보여, 이후 리팩토링을 해보고 싶다.
+ Computer Vision에 비하여, NLP의 난이도가 상당한 것으로 보이지만, 그만큼 깊게 공부하였을 때 의미가 있는 학문으로 판단된다. 다만 현재의 목표를 모델 설계나 정확도를 높이는 데에 집중하기 보다는 **현재 개발되어 있는 우수하고 효율적인 모델을 선적용하여 사회문제를 해결하는 방향성이 효율적이라 판단된다.**
    - 현재 본인의 학습 수준은 주니어 레벨이고, NLP 모델에 대한 설계는 알고리즘까지 설계 가능한 Data Scientist가 진행하는 것이라 판단된다.
    - 따라서 2-3개월차가 되었을 즈음에는 모델 변경 및 적용까지 가능한 수준으로 만들어야겠다는 판단을 하게 되었고, 다만 현재로서는 역량이 그에 미치지 못하는 것 같아 이 부분은 프로젝트를 진행하며 아쉬운 점으로 남는다.
    - 현재로선 데이터 수집 - 전처리 - 예측 모델 적용 - 결과 비교 및 (부분 튜닝) 과정에 집중할 것이다.